In [1]:
import requests
import json
import pandas as pd
import os

from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv('API_KEY')
API_KEY_SECRET = os.getenv('API_KEY_SECRET')
BEARER_TOKEN = os.getenv('BEARER_TOKEN')

url = "https://api.twitter.com/2/tweets/search/recent?query=oscars2022%20lang%3Aen%20-is%3Aretweet%20-has%3Alinks&max_results=100&tweet.fields=created_at"

payload={}
headers = {
  'Authorization': f'Bearer {BEARER_TOKEN}',
  'Cookie': 'guest_id=v1%3A164853211096256691; guest_id_ads=v1%3A164853211096256691; guest_id_marketing=v1%3A164853211096256691; personalization_id="v1_R51PgaLxBcdDSniM7AfLhA=="'
}

# get tweets from the API
tweets = requests.request("GET", url, headers=headers, data=payload)
print(tweets.json())

{'data': [{'created_at': '2022-04-01T05:58:28.000Z', 'id': '1509772163406049282', 'text': '#WillSmith apology is fake af! Lost all respect for him! Fuckin whiney wanker! #Jada loves her bald head.. quit making lame-ass excuses for thug behavior at a black tie event. Get your ass in check dude! The devil is you, not Chris! I hope you get black listed! #Oscars2022 😒🖕'}, {'created_at': '2022-04-01T05:55:04.000Z', 'id': '1509771306337771526', 'text': "What if they made a movie where the audience is the world population, Will Smith was America, Chris Rock was Russia &amp; Will Smith's wife was Ukraine. Who would be the absent law enforcers &amp; what would be the name of the movie? #Oscars2022."}, {'created_at': '2022-04-01T05:47:47.000Z', 'id': '1509769475016511509', 'text': "@amyschumer was traumatized by the violence she saw at the Oscars. White people are always traumatized by violence that's in close proximity to them. They however can know that black people live with the threat of vio

In [2]:
json_tweets = tweets.json()
json_data = json_tweets['data']

In [3]:
created_at_data = [d['created_at'] for d in json_data]
id_data = [d['id'] for d in json_data]
text_data = [d['text'] for d in json_data]

tweets_df = pd.DataFrame({'created_at': created_at_data, 'id': id_data, 'text': text_data})
tweets_df.shape

(100, 3)

In [4]:
tweets_df.head(10)

,created_at,id,text
0,2022-04-01T05:58:28.000Z,1509772163406049282,#WillSmith apology is fake af! Lost all respec...
1,2022-04-01T05:55:04.000Z,1509771306337771526,What if they made a movie where the audience i...
2,2022-04-01T05:47:47.000Z,1509769475016511509,@amyschumer was traumatized by the violence sh...
3,2022-04-01T05:47:19.000Z,1509769358406389761,Miss Ma’am now playing the victim card after h...
4,2022-04-01T05:44:16.000Z,1509768587699007517,.... and wonder how many will go for the compl...
5,2022-04-01T05:42:27.000Z,1509768133204058120,The world needs more #comedians than it needs ...
6,2022-04-01T05:39:41.000Z,1509767436601634825,How as a comedian does Will get mad at a joke....
7,2022-04-01T05:38:11.000Z,1509767056505245698,May be this is the first time #ChrisRock told ...
8,2022-04-01T05:35:34.000Z,1509766400260419643,I’m so tired of hearing about this damn slap b...
9,2022-04-01T05:21:16.000Z,1509762799731453972,@cnn gotta ot of karens exposing thereself cov...
